In [1]:
import tensorflow as tf
from tensorflow.keras.applications.densenet import DenseNet169
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Define the input shape and number of classes
input_shape = (224, 224, 3)
num_classes = 1

In [3]:
# Define the data augmentation and normalization transforms
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=90,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load the dataset
train_dataset = train_datagen.flow_from_directory(
    directory='/kaggle/input/melanoma10000/Melanoma10000/Original/Train',
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='binary'
)

test_dataset = test_datagen.flow_from_directory(
    directory='/kaggle/input/melanoma10000/Melanoma10000/Original/Test',
    #input_shape is a tuple containing the dimensions of the input images, which is (224, 224, 3) in this case. The [:2] indexing is used to select the first two dimensions of the tuple, which are the height and width of the images.
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='binary'
)
#Since we have set class_mode='binary', the labels will be 0 for one class and 1 for the other class.

Found 9605 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [4]:
# Load the pre-trained DenseNet-169 model
base_model = DenseNet169(weights='imagenet', include_top=False, input_shape=input_shape)

# Add a new fully-connected layer for classification
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dense(num_classes, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=x)

51877672/51877672 [==============================] - 0s 0us/step


In [6]:
# Freeze the pre-trained layers and compile the model
for layer in base_model.layers:
    layer.trainable = False

model.compile(
    optimizer=Adam(lr=0.0001),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

In [7]:
# Train the model
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=test_dataset
)

Epoch 1/10
301/301 [==============================] - 208s 645ms/step - loss: 1.0005 - accuracy: 0.8447 - val_loss: 0.3057 - val_accuracy: 0.8880
Epoch 2/10
301/301 [==============================] - 120s 398ms/step - loss: 0.2962 - accuracy: 0.8877 - val_loss: 0.2820 - val_accuracy: 0.8950
Epoch 3/10
301/301 [==============================] - 119s 394ms/step - loss: 0.2421 - accuracy: 0.9031 - val_loss: 0.2876 - val_accuracy: 0.8700
Epoch 4/10
301/301 [==============================] - 118s 391ms/step - loss: 0.2333 - accuracy: 0.9037 - val_loss: 0.2522 - val_accuracy: 0.9040
Epoch 5/10
301/301 [==============================] - 118s 392ms/step - loss: 0.2244 - accuracy: 0.9082 - val_loss: 0.2376 - val_accuracy: 0.9050
Epoch 6/10
301/301 [==============================] - 118s 393ms/step - loss: 0.2267 - accuracy: 0.9087 - val_loss: 0.2270 - val_accuracy: 0.9080
Epoch 7/10
301/301 [==============================] - 119s 394ms/step - loss: 0.2104 - accuracy: 0.9136 - val_loss: 0.2273 -

In [8]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_dataset)
print('Test accuracy:', test_acc)

32/32 [==============================] - 3s 95ms/step - loss: 0.2256 - accuracy: 0.9060
Test accuracy: 0.906000018119812


In [9]:
# Save the model
model.save('my_model.h5')

In [10]:
#load h5 module
model=tf.keras.models.load_model('my_model.h5')
tflite_converter = tf.lite.TFLiteConverter.from_keras_model(model)

#convert
tflite_model = tflite_converter.convert()
open("tf_lite_model.tflite", "wb").write(tflite_model)

217028216